<a href="https://colab.research.google.com/github/ZohaibCodez/openai-agents-chainlit-lab/blob/main/01_hello_agent/streaming/hello_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Google's Gemini using run_streamed with OpenAI Agents SDK in Colab

## Installing OpenAI Agents SDk

In [1]:
!pip install -Uq openai-agents

## Making Jupyter Notebook capable of running asynchronous functions like our RunLoopEvent(Runner.run_streamed)

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Running Google's GEMINI with OpenAI Agents SDK

In [4]:
from agents import Agent,Runner,AsyncOpenAI,OpenAIChatCompletionsModel,RunResult
from agents.run import RunConfig
from google.colab import userdata

In [5]:
gemini_api_key = userdata.get("GOOGLE_API_KEY")

if not gemini_api_key:
  raise ValueError("GEMINI_API_KEY is not set. Please ensure it is set")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
    )

model = OpenAIChatCompletionsModel(
    model = "gemini-2.0-flash",
    openai_client = external_client
)

run_config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

## Hello World Code | Run

In [10]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent

async def main():
  agent:Agent = Agent(name="Assistant",instructions = "You are an agent - please keep going until the user’s query is completely resolved, before ending your turn and yielding back to the user. Only terminate your turn when you are sure that the problem is solved.",model=model)

  result:RunResult =Runner.run_streamed(starting_agent=agent,input="Hello...! My name is Zohaib",run_config=run_config)
  async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
      print(event.data.delta,end="",flush=True)

  new_input = result.to_input_list() + [{"role": "user", "content": "What is my name?"}]

  result2:RunResult =Runner.run_streamed(starting_agent=agent,input=new_input,run_config=run_config)
  async for event in result2.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
      print(event.data.delta,end="",flush=True)

  new_input2 = result2.to_input_list() + [{"role": "user", "content": "Tell me 5 jokes about coding"}]

  result2:RunResult =Runner.run_streamed(starting_agent=agent,input=new_input2,run_config=run_config)
  async for event in result2.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
      print(event.data.delta,end="",flush=True)

asyncio.run(main())

Hello Zohaib! It's nice to meet you. How can I help you today?
Your name is Zohaib.
Okay, Zohaib, here are 5 coding jokes for you:

1.  **Why do programmers prefer dark mode?** Because light attracts bugs!

2.  **Why was the JavaScript developer sad?** Because they didn't Node how to Express themselves.

3.  **Why did the programmer quit his job?** Because he didn't get arrays.

4.  **There are 10 types of people in the world:** Those who understand binary, and those who don't.

5.  **Debugging:** Removing the needles from the haystack.

Hope you enjoyed those, Zohaib! Let me know if you'd like to hear more.
